In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import requests
from PIL import Image
import time
import torch
import h5py
import glob
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import BertTokenizer, TFBertModel
import torchtext.vocab as vocab
import argparse
import random
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import Dataset
from torchvision import transforms
from transformers import BertTokenizer, TFBertModel
from tqdm.notebook import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
train_dataset = h5py.File('/kaggle/input/443-datasets/eee443_project_dataset_train.h5','r')

train_cap = train_dataset["train_cap"][()]
train_imid = train_dataset["train_imid"][()]
train_url = train_dataset["train_url"][()]
train_ims = train_dataset["train_ims"][()]
word_code = train_dataset["word_code"][()]


test_dataset = h5py.File('/kaggle/input/443-datasets/eee443_project_dataset_test.h5','r')
test_cap = test_dataset["test_caps"][()]
test_imid = test_dataset["test_imid"][()]
test_url=test_dataset["test_url"][()]

In [ ]:
words = pd.DataFrame(word_code)
words = words.sort_values(0, axis=1)
array_of_words = np.asarray(words.columns)

word_dict = {}
index_dict = {}
for i in range(len(array_of_words)):
 w = array_of_words[i]
 word_dict[w] = i
 index_dict[i] = w

In [ ]:
train_img_dir="/kaggle/input/443-train-images/443images"
test_img_dir="/kaggle/input/443-test-images/443test_images"

In [ ]:
train_imid2=np.load("/kaggle/input/cleaned-data/trainimid.npy") 
train_cap2=np.load("/kaggle/input/cleaned-data/traicap.npy")
test_imid2=np.load("/kaggle/input/cleaned-data/testimid.npy")
test_cap2=np.load("/kaggle/input/cleaned-data/testcap.npy")

In [ ]:

class Create_Dataset(Dataset):

  def __init__(self, imid, cap, dir_path, **kwargs):

   self.dir_path = dir_path
   self.sample_n = len(imid)

   self.imid = imid
   self.cap = cap

  def __len__(self):

    return self.sample_n

  def __getitem__(self, ind):

     imgpath = self.dir_path + "/" + str(self.imid[ind]) + ".png"
     self.image = Image.open(imgpath).convert("RGB")

     data_tensor = self.ImageToTensor(self.image)

     return (data_tensor, self.cap[ind])

  def ImageToTensor(self, img):

    transform_with_resize = transforms.Compose([
         transforms.Resize(64),
         transforms.CenterCrop(64),
         transforms.ToTensor(),
         transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))
    ])
    get_image = transforms.Compose([transforms.ToPILImage()])
    img_rsz = transform_with_resize(img)
    tns_to_img = get_image(img_rsz)

    if img_rsz.shape[0]>3:
     img_rsz = img_rsz[:3]
    return img_rsz

  def get_splits(self):

   n_tra = int(0.85 * self.sample_n)
   n_val = self.sample_n-n_tra
    
   self.get_train, self.get_val = torch.utils.data.random_split(self, [n_tra, n_val])


In [ ]:
train_set = Create_Dataset(train_imid2, train_cap2, train_img_dir)
test_set = Create_Dataset(test_imid2, test_cap2, test_img_dir)
torch.cuda.empty_cache()

In [ ]:
train_set.get_splits()
validation_data = train_set.get_val
train_data = train_set.get_train
print(len(train_data))
print(len(validation_data))

In [ ]:
torch.cuda.empty_cache()
train_dataloader = torch.utils.data.DataLoader(

  dataset=train_data,

  batch_size=64,

  shuffle=True,

  num_workers=0,

  drop_last = True

)

In [ ]:
torch.cuda.empty_cache()
val_dataloader = torch.utils.data.DataLoader(

    dataset=validation_data,

    batch_size=64,

    shuffle=False,

    num_workers=0,

    drop_last = True

)

In [ ]:
torch.cuda.empty_cache()

test_dataloader = torch.utils.data.DataLoader(

    dataset=test_set,

    batch_size=64,

    shuffle=False,

    num_workers=0,

    drop_last = True

)
print(len(test_dataloader))

In [ ]:
name_bert="prajjwal1/bert-small"

tokenizer=BertTokenizer.from_pretrained(name_bert)

model = TFBertModel.from_pretrained(name_bert,from_pt=True)

def enc_sntcs(sntc):

 cap_list = []

 for i in sntc:

     cap_list.append(array_of_words[i])

 s = " ".join(cap_list)



 encd = tokenizer.batch_enc_plus(

                [s],

                max_length=512,

                pad_to_max_length=False,

                return_tensors="pt",

                truncation=True,

            )



 input_ids = np.array(encd["input_ids"], dtype="int32")

 out = model(input_ids)

 seq, pool = out[:2]

 return pool[0]

In [ ]:
def enc_sntcs_batch(sntcs):
 enc_sntcs = []
 for sntc in sntcs:
  enc_sntcs.append(enc_sntcs(sntc))
 return torch.tensor(np.array(enc_sntcs))

In [ ]:
rand_index=443 ## for reproduction
random.seed(rand_index)
torch.manual_seed(rand_index)

In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

workers = 1
batch_size = 64
image_size = 64
nc = 3
nz = 128
ngf = 64
ndf = 64
num_epochs = 1
lr = 2e-4
beta1 = 0.5
ngpu = 2

In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

def weights_init(m):

  classname = m.__class__.__name__

  if classname.find('Conv') != -1:
    nn.init.xavier_uniform_(m.weight.data)

  elif classname.find("Linear") != -1:
    nn.init.xavier_uniform_(m.weight.data)
    nn.init.constant_(m.bias.data, 0)

  elif classname.find('BatchNorm') != -1:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)




In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

class Generator(nn.Module):
  def __init__(self,ngpu):
    super(Generator,self).__init__()
    self.ngpu = ngpu
    self.proj_dim_emb = 128
    self.noise_dim = nz
    self.latent_dim = self.noise_dim + self.proj_dim_emb
    self.epsilon = torch.randn(batch_size,128,1,1,device=device)
    self.project = nn.Sequential (
                  nn.Linear(512,self.proj_dim_emb),
                  nn.BatchNorm1d(num_features=self.proj_dim_emb),
                  nn.LeakyReLU(negative_slope=0.2,inplace = True))
    self.main = nn.Sequential(
      nn.ConvTranspose2d(self.latent_dim, ngf * 8, 4, 1, 0, bias=False),
      nn.BatchNorm2d(ngf * 8),
      nn.ReLU(True),
      nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 4),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 2),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf),
      nn.ReLU(True),
      nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
      nn.Tanh()
      )

  def forward(self, inp,n):

    emb = self.project(inp).unsqueeze(2).unsqueeze(3)
    lat = torch.concat([emb,n],1)
    
    return self.main(lat)

In [ ]:
netG = Generator(ngpu).to(device)
if (device.type == 'cuda') and (ngpu > 1):
  netG = nn.DataParallel(netG, list(range(ngpu)))
# netG.apply(weights_init)  ## Use it for first epoch
netG.load_state_dict(torch.load("/kaggle/input/dcganep1/netG_0_bs_64.pth")) ## for the next epochs

In [ ]:

class Discriminator(nn.Module):
  def __init__(self, ngpu):
    super(Discriminator, self).__init__()
    self.ngpu = ngpu
    self.dim_emb = 512
    self.proj_dim_emb = 128
    self.main = nn.Sequential(
      nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf * 2),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf * 4),
      nn.LeakyReLU(0.2, inplace=True),
      nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ndf * 8),
      nn.LeakyReLU(0.2, inplace=True)   )

    self.proj = nn.Sequential(
        nn.Linear(self.dim_emb,self.proj_dim_emb),
        nn.BatchNorm1d(self.proj_dim_emb),
        nn.LeakyReLU(negative_slope=0.2, inplace=True)   )

    self.aug = nn.Sequential(
        nn.Conv2d(8*ndf  + self.proj_dim_emb, 1, 4, 1, 0, bias=False),
        nn.Sigmoid()  )



  def forward(self, inpimg,capt):
    intimg= self.main(inpimg)
    emb_proj = self.proj(capt)
    emb_rep = emb_proj.repeat(4,4,1,1).permute(2,3,0,1)
    conc_hid = torch.concat([intimg,emb_rep],1)
    out = self.aug(conc_hid)
    return out.view(-1,1).squeeze(1)

In [ ]:
netD = Discriminator(ngpu).to(device)
if (device.type == 'cuda') and (ngpu > 1):
 netD = nn.DataParallel(netD, list(range(ngpu)))
#netD.apply(weights_init)  ## Use for first epoch
netD.load_state_dict(torch.load("/kaggle/input/dcganep1/netD_0_bs_64.pth")) ## for the next epochs

In [ ]:
criterion = nn.BCELoss()
fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)
real_label = 1.
fake_label = 0.
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

img_list = []
G_losses = []
D_losses = []
G_losses_val = []
D_losses_val = []
iters = 0


for epoch in range(num_epochs):
  i=0

  for imgs, txt_emb in tqdm(train_dataloader):

    netD.zero_grad()
    img = imgs.to(device)
    capts = enc_sntcs_batch(txt_emb).to(device)
    b_size = img.size(0)
    label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
    capts = capts.to(torch.float32)
    output = netD(img,capts).view(-1)
    errD_real = criterion(output, label)
    errD_real.backward()
    D_x = output.mean().item()
    noise = torch.randn(b_size, nz,1,1, device=device)
    fake = netG(capts,noise)
    label.fill_(fake_label)
    output = netD(fake.detach(),capts).view(-1)
    errD_fake = criterion(output, label)
    errD_fake.backward()
    D_G_z1 = output.mean().item()
    errD = errD_real + errD_fake
    optimizerD.step()

    netG.zero_grad()
    label.fill_(real_label) 
    output = netD(fake,capts).view(-1)
    errG = criterion(output, label)
    errG.backward()

    D_G_z2 = output.mean().item()
    optimizerG.step()

    if i % 50 == 0:
        print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
           % (epoch, num_epochs, i, len(train_dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))


    G_losses.append(errG.item())
    D_losses.append(errD.item())

    if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_dataloader)-1)):   
        with torch.no_grad():
            fake = netG(capts,fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
    i += 1
    iters += 1

    
  torch.save(netG.state_dict(),"/kaggle/working/netG_1_bs.pth")
  torch.save(netD.state_dict(),"/kaggle/working/netD_1_bs.pth")


In [ ]:
np.save("gloss1.npy",np.array(G_losses))
np.save("dloss1.npy",np.array(D_losses))

In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
### https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
real_batch = next(iter(test_dataloader))
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64],
                            padding=5, normalize=True).cpu(),(1,2,0)))
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()